In [8]:
!pip install gensim -i https://pypi.tuna.tsinghua.edu.cn/simple

# data pre

In [1]:
from tqdm import tqdm
import pandas as pd

data_test = pd.read_csv('test_.csv')
# test = data_test.drop(columns=['ID'])

data_train = pd.read_csv('train_.csv').sort_values(by=['肝炎'])#.tail(5000)
data_train = data_train[ ~ data_train['肝炎'].isin([-1])] 
# train = data_train.drop(columns=['ID','肝炎'])
train_ = data_train.drop(columns=['肝炎'])

label = data_train['肝炎']

data = train_.append(data_test)

data.columns = ['Age','Gender','Area','Weight','Height','Body_mass_index',
                'Obesity_waistline','Waist','Highest_blood_pressure','Minimum_blood_pressure',
                'Good_Cholesterol','Bad_Cholesterol','Total_Cholesterol','Blood_lipid_abnormality',
                'PVD','Sports_activities','Education','Unmarried','Revenue','Source_of_care',
                'Poor_vision','Drinking','Hypertension','Family_hypertension','Diabetes',
                'Family_diabetes','Family_hepatitis','Chronic_fatigue','ALF','ID']

train = data.head(train_.shape[0])
test = data.tail(data_test.shape[0])

pre1 = pd.read_csv('/home/kesci/work/test_pre.csv').dropna(axis=0,how='any')
pre1 = pre1['id'].astype(int)
pre2 = pd.read_csv('/home/kesci/work/train_pre.csv').dropna(axis=0,how='any')
pre2 = pre2['id'].astype(int)
pre3 = pre1.append(pre2).values.flatten().tolist()

now = test['ID'].astype(int).values.flatten().tolist()
know_test_id = list(set(now)&set(pre3))
need_test_id = list(set(now).difference(set(pre3)))

pre1_ = pd.read_csv('/home/kesci/work/test_pre.csv')
pre2_ = pd.read_csv('/home/kesci/work/train_pre.csv')
pre3_ = pre1_.append(pre2_)

pre_know_id = pd.DataFrame({'id':know_test_id})
pre_need_id = pd.DataFrame({'ID':need_test_id})

pre_konw = pd.merge(pre_know_id, pre3_, on=['id'],how='left')

pre_konw.columns = ['ID','hepatitis']

val_online = pd.merge(pre_konw['ID'], test, on=['ID'],how='left').drop(columns=['ID'])
val_online_label = pre_konw['hepatitis']
test = test.drop(columns=['ID'])
train = train.drop(columns=['ID'])

## w2v

In [2]:
data = train.append(test)
data = data.append(val_online).astype(str)
fea_list = ['Age','Gender','Area','Weight','Height','Body_mass_index',
            'Obesity_waistline','Waist','Highest_blood_pressure','Minimum_blood_pressure',
            'Good_Cholesterol','Bad_Cholesterol','Total_Cholesterol','Blood_lipid_abnormality',
            'PVD','Sports_activities','Education','Unmarried','Revenue','Source_of_care',
            'Poor_vision','Drinking','Hypertension','Family_hypertension','Diabetes',
            'Family_diabetes','Family_hepatitis','Chronic_fatigue','ALF']
for i in fea_list:
    data[i] = data[i]+'_'+i
data.to_csv('data_str.csv',sep=' ',index=0)

In [3]:
from gensim.models import word2vec
from gensim.models.callbacks import CallbackAny2Vec

print('LineSentence start')
sentences = word2vec.LineSentence('data_str.csv') 
print('Word2Vec start')
model = word2vec.Word2Vec(sentences , size=128, window=64, sg=0, hs=1, min_count=1, iter=10, workers=-1)
print('save start')
model.save('word2vec_model.model')
model.wv.save('word2vec_kv.kv')
print('Done')


LineSentence start
Word2Vec start


/opt/conda/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


save start


/opt/conda/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/opt/conda/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Done


In [4]:
from gensim.models import KeyedVectors
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
def get_embedding():
        path = f'word2vec_kv.kv'
        wv = KeyedVectors.load(path, mmap='r')
        feature_tokens = list(wv.vocab.keys())
        tokenizer = Tokenizer(num_words=len(feature_tokens))
        tokenizer.fit_on_texts(feature_tokens)
        embedding_dim = 128
        embedding_matrix = np.random.randn(
            len(feature_tokens)+1, embedding_dim)
        for feature in feature_tokens:
            embedding_vector = wv[feature]
            if embedding_vector is not None:
                index = tokenizer.texts_to_sequences([feature])[0][0]
                embedding_matrix[index] = embedding_vector
        return embedding_matrix

embedding_matrix = get_embedding()

/opt/conda/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


## input

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec, KeyedVectors
import numpy as np

In [6]:
path = f'word2vec_kv.kv'
wv = KeyedVectors.load(path, mmap='r')
feature_tokens = list(wv.vocab.keys())
tokenizer = Tokenizer(num_words=len(feature_tokens), filters='')
tokenizer.fit_on_texts(feature_tokens)

feature_seq = []
with open('data_str.csv') as f:
    for text in f:
        feature_seq.append(text.strip())

sequences = tokenizer.texts_to_sequences(feature_seq)
data_input = pad_sequences(
    sequences, maxlen=40, padding='post')

trans_train = data_input[:train.shape[0]]

trans_test_ = data[test.shape[0]+val_online.shape[0]:]
trans_test = trans_test_[:test.shape[0]]

trans_val_online = trans_test_[(val_online.shape[0]):]


/opt/conda/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


# model

In [8]:
from layers import Add, LayerNormalization
from layers import MultiHeadAttention, PositionWiseFeedForward
from layers import PositionEncoding

from itertools import chain

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import multi_gpu_model
from tensorflow.keras import optimizers, layers, losses
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Concatenate, GlobalMaxPooling1D, Flatten
from tensorflow.keras.backend import concatenate
from gensim.models import Word2Vec, KeyedVectors

import pandas as pd


In [11]:
def M(emb):


    feed_forward_size_trans = 2048
    max_seq_len_trans = 40
    model_dim_trans = 128

    input_trans = Input(shape=(max_seq_len_trans,), name='input_trans_layer')
   
    x = Embedding(input_dim=emb.shape[0],
                   output_dim=128,
                   weights=[emb],
                   trainable=False,
                   input_length=40,
                   mask_zero=True)(input_trans)

   
    encodings = PositionEncoding(model_dim_trans)(x)
    encodings = Add()([x, encodings])

    # encodings = x
    masks = tf.equal(input_trans, 0)

    # (bs, 100, 128*2)
    attention_out = MultiHeadAttention(4, 32)(
        [encodings, encodings, encodings, masks])

    # Add & Norm
    attention_out += encodings
    attention_out = LayerNormalization()(attention_out)
    # Feed-Forward
    ff = PositionWiseFeedForward(model_dim_trans, feed_forward_size_trans)
    ff_out = ff(attention_out)
    # Add & Norm
    ff_out += attention_out
    encodings = LayerNormalization()(ff_out)
    encodings = GlobalMaxPooling1D()(encodings)
    encodings = Dropout(0.2)(encodings)

    output = Dense(2, activation='softmax', name='prediction_layer')(encodings)
  
    
    model = Model(inputs=[input_trans], 
                outputs=[output])

    model.compile(
        optimizer=optimizers.Adam(2.5e-4),
        loss={'prediction_layer': losses.binary_crossentropy},
        metrics=['AUC'])
    return model



In [12]:
model = M(embedding_matrix)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Poor_vision (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
PVD (InputLayer)                [(None, 1)]          0                                            
__________________________________________________________________________________________________
Obesity_waistline (InputLayer)  [(None, 1)]          0                                            
__________________________________________________________________________________________________
Family_diabetes (InputLayer)    [(None, 1)]          0                                            
_______________________________________________________________________________________

In [13]:
input_train = {'input_trans_layer': trans_train,}
model.fit(input_train,
        {'prediction_layer':label},
        # validation_split=0.3,
        validation_data=(deepfm_val_online, val_online_label),
        epochs=5,
        batch_size=100,)

(5984, 40)
Epoch 1/5


/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


5984/5984 [==============================] - 43s 7ms/step - loss: 0.2003 - auc: 0.8216
Epoch 2/5
5984/5984 [==============================] - 41s 7ms/step - loss: 0.1845 - auc: 0.8612
Epoch 3/5
5984/5984 [==============================] - 41s 7ms/step - loss: 0.1834 - auc: 0.8622
Epoch 4/5
5984/5984 [==============================] - 41s 7ms/step - loss: 0.1820 - auc: 0.8652
Epoch 5/5
5984/5984 [==============================] - 40s 7ms/step - loss: 0.1807 - auc: 0.8646


In [ ]:
input_test = {'input_trans_layer': trans_test,}
ans_mtx = model.predict(input_test,
                        batch_size=300)
